<img title="GitHub Octocat" src='./img/Octocat.jpg' style='height: 60px; padding-right: 15px' alt="Octocat" align="left" height="60"> This notebook is part of a GitHub repository: https://github.com/pessini/moby-bikes
<br>MIT Licensed
<br>Author: Leandro Pessini

# <p style="font-size:100%; text-align:left; color:#444444;">Data Wrangling</p>

# <p style="font-size:100%; text-align:left; color:#444444;">Table of Contents:</p>
* [1. Datasets](#1)
  * [1.1 Rentals Data - Moby Bikes](#1.1)
  * [1.2 Weather Data - Met Éireann](#1.2)
* [2. Feature Engineering](#2)
  * [2.1 Target variable distribution](#2.1)
  * [2.2 Missing values](#2.2)
  * [2.3 Exploratory Analysis](#2.3)
  * [2.4 Features Importance](#2.4)

In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

<a id="1"></a>
# <p style="font-size:100%; text-align:left; color:#444444;">1- Datasets</p>

Dataset provided by [Moby Bikes](https://data.gov.ie/dataset/moby-bikes) through a public [API](https://data.smartdublin.ie/mobybikes-api). 

Dataset provided by [Met Éireann](https://www.met.ie/) through a public [API](https://data.gov.ie/organization/meteireann).


[Met Éireann Weather Forecast API](https://data.gov.ie/dataset/met-eireann-weather-forecast-api/resource/5d156b15-38b8-4de9-921b-0ffc8704c88e)

<a id="1.1"></a>

## Rentals Data - Moby Bikes

### MongoDB vs DynamoDB

In [50]:
historical_data = pd.read_csv('../data/raw/historical_data.csv')

In [51]:
historical_data.columns = historical_data.columns.str.lower()
historical_data.head()

,harvesttime,bikeid,battery,bikeidentifier,biketypename,ebikeprofileid,ebikestateid,isebike,ismotor,issmartlock,lastgpstime,lastrentalstart,latitude,longitude,spikeid
0,2021-04-01 00:00:03,5,7.0,1,DUB-General,1,2,True,False,False,2021-03-31 23:41:40,2021-03-30 19:18:18,53.3091,-6.21643,1
1,2021-04-01 00:00:03,6,16.0,2,DUB-General,1,2,True,False,False,2021-03-31 23:55:41,2021-03-31 10:31:13,53.3657,-6.32249,2
2,2021-04-01 00:00:03,7,66.0,3,DUB-General,4,2,True,False,False,2021-03-31 23:42:04,2021-03-30 13:07:19,53.2799,-6.14497,3
3,2021-04-01 00:00:03,8,48.0,4,DUB-General,1,2,True,False,False,2021-03-31 23:52:26,2021-03-30 12:43:17,53.2891,-6.11378,4
4,2021-04-01 00:00:03,9,-6.0,5,DUB-General,1,2,True,False,False,2021-03-31 23:50:20,2021-03-29 22:37:58,53.2928,-6.13014,5


In [52]:
print('Total number of rows: {}'.format(historical_data.shape[0]))
print('Total number of columns: {}'.format(historical_data.shape[1]))


Total number of rows: 1667841
Total number of columns: 15


In [53]:
historical_data.isnull().sum()

harvesttime            0
bikeid                 0
battery            42341
bikeidentifier         0
biketypename           0
ebikeprofileid         0
ebikestateid           0
isebike                0
ismotor                0
issmartlock            0
lastgpstime            0
lastrentalstart        0
latitude               0
longitude              0
spikeid                0
dtype: int64

<a id="1.2"></a>

## Weather Data - Met Éireann

Regarding the weather data there are two important decisions to deal with.

- One is about from **which station** the **historical data will be collected**;
-  and the other one is about the **frequency of data**, which can be **hourly or daily**.

### Station Name: **PHOENIX PARK**

In [54]:
# Hourly data from Phoenix Park Station
phoenixpark_weather_hourly = pd.read_csv('../data/raw/hly175.csv')
phoenixpark_weather_hourly.head()

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
0,16-aug-2003 01:00,0,0.0,0,9.2,0,8.9,8.5,11.1,95,1021.9
1,16-aug-2003 02:00,0,0.0,0,9.0,0,8.7,8.5,11.1,96,1021.7
2,16-aug-2003 03:00,0,0.0,0,8.2,0,8.0,7.7,10.5,96,1021.2
3,16-aug-2003 04:00,0,0.0,0,8.4,0,8.1,7.9,10.7,97,1021.2
4,16-aug-2003 05:00,0,0.0,0,7.7,0,7.5,7.3,10.2,97,1021.1


Source: [https://data.gov.ie/dataset/phoenix-park-hourly-data](https://data.gov.ie/dataset/phoenix-park-hourly-data)

In [55]:
# Daily data from Phoenix Park Station
phoenixpark_weather_daily = pd.read_csv('../data/raw/dly175.csv')
phoenixpark_weather_daily.head()

,date,ind,maxtp,ind.1,mintp,igmin,gmin,ind.2,rain,cbl,soil
0,16-aug-2003,0,20.1,0,7.5,4,,0,0.0,1013.7,18.565
1,17-aug-2003,0,21.3,0,11.6,0,7.5,0,1.1,1007.5,18.280
2,18-aug-2003,0,20.3,0,8.5,0,4.3,0,0.0,1008.8,17.825
3,19-aug-2003,0,19.9,0,11.3,0,7.7,0,0.0,1014.3,18.138
4,20-aug-2003,0,21.5,0,10.8,0,6.9,0,0.0,1013.6,18.432


Source: [https://data.gov.ie/dataset/phoenixpark-daily-data](https://data.gov.ie/dataset/phoenixpark-daily-data)

### Station Name: **DUBLIN AIRPORT**

In [56]:
# Hourly data from Dublin Airport Station
dublin_airport_weather_hourly = pd.read_csv('../data/raw/hly532.csv')
dublin_airport_weather_hourly.head()

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
0,01-jan-1992 00:00,0,0.0,0,8.4,0,6.6,4.3,8.3,75,...,2,23,2,210,2,11,0.0,25000,999,3
1,01-jan-1992 01:00,0,0.0,0,8.6,0,6.6,4.0,8.1,73,...,2,23,2,220,2,11,0.0,25000,100,6
2,01-jan-1992 02:00,0,0.0,0,9.0,0,7.0,4.5,8.4,73,...,2,22,2,220,2,11,0.0,25000,100,6
3,01-jan-1992 03:00,0,0.0,0,9.5,0,7.4,4.8,8.6,73,...,2,22,2,220,2,11,0.0,25000,25,6
4,01-jan-1992 04:00,0,0.0,0,9.5,0,7.4,4.8,8.6,73,...,2,23,2,230,2,11,0.0,25000,25,6


In [57]:
print('Total number of rows: {}'.format(dublin_airport_weather_hourly.shape[0]))

Total number of rows: 264409


Source: [https://data.gov.ie/dataset/dublin-airport-hourly-data](https://data.gov.ie/dataset/dublin-airport-hourly-data)

### Phoenix Park Station vs Dublin Aiport Station
Geographically, the station at Phoenix Park would be the most suitable choice but unfortunately, they do not collect Wind information which in Ireland plays an important role when deciding to go cycling or not. For those who are not familiar with Irish weather, it rains a lot and mostly we do not have much choice about it but the wind is something that can prevent you go outside or choosing a different kind of transportation. Heavy rain is not that common, though.

### Hourly vs Daily data
Daily data, to the business, could make more sense but because the weather is so unpredictable in Ireland (it can completely change in an hour), the best option would be hourly data if looking at a historical perspective. For simplicity and better planning, we can always aggregate the predicted results by day.

In [58]:
# transforming the date columns in weather data to datetime
dublin_airport_weather_hourly['date'] = pd.to_datetime(dublin_airport_weather_hourly['date'])
dublin_airport_weather_hourly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264409 entries, 0 to 264408
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    264409 non-null  datetime64[ns]
 1   ind     264409 non-null  int64         
 2   rain    264409 non-null  float64       
 3   ind.1   264409 non-null  int64         
 4   temp    264409 non-null  float64       
 5   ind.2   264409 non-null  int64         
 6   wetb    264409 non-null  float64       
 7   dewpt   264409 non-null  float64       
 8   vappr   264409 non-null  object        
 9   rhum    264409 non-null  object        
 10  msl     264409 non-null  float64       
 11  ind.3   264409 non-null  int64         
 12  wdsp    264409 non-null  int64         
 13  ind.4   264409 non-null  int64         
 14  wddir   264409 non-null  object        
 15  ww      264409 non-null  int64         
 16  w       264409 non-null  int64         
 17  sun     264409 non-null  floa

In [59]:
dublin_airport_weather_hourly.tail()

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
264404,2022-02-28 20:00:00,3,0.0,0,2.2,0,1.4,0.0,6.1,86,...,2,7,2,300,2,11,0.0,30000,999,1
264405,2022-02-28 21:00:00,3,0.0,0,1.1,0,0.6,-0.3,6.0,90,...,2,5,2,290,2,11,0.0,30000,999,1
264406,2022-02-28 22:00:00,3,0.0,0,0.0,1,-0.3,-1.0,5.7,94,...,2,6,2,290,2,11,0.0,30000,999,1
264407,2022-02-28 23:00:00,3,0.0,0,0.2,1,-0.1,-0.7,5.8,94,...,2,6,2,290,2,11,0.0,30000,999,1
264408,2022-03-01 00:00:00,3,0.0,1,-0.2,1,-0.4,-0.9,5.8,96,...,2,6,2,280,2,11,0.0,30000,999,1


### Sampling

In [60]:
start_date_hist = datetime(2021, 2, 1) # first day
end_date_hist = datetime(2022, 3, 1) # last day used as historical data

In [61]:
recent_dubairport_data = dublin_airport_weather_hourly.copy()
recent_dubairport_data = recent_dubairport_data[(recent_dubairport_data.date >= start_date_hist) & (recent_dubairport_data.date <= end_date_hist)]
len(dublin_airport_weather_hourly), len(recent_dubairport_data)

(264409, 9433)

In [62]:
columns_to_drop = ['ind','ind.1','ind.2','ind.3','vappr','msl','ind.4','wddir','ww','w','sun','vis','clht','clamt']
weather_data = recent_dubairport_data.drop(columns=columns_to_drop)
weather_data.to_csv('../data/interim/hist_weather_data.csv', index=False)

In [63]:
weather_data.head()

,date,rain,temp,wetb,dewpt,rhum,wdsp
254976,2021-02-01 00:00:00,0.0,3.3,3.1,2.8,97,8
254977,2021-02-01 01:00:00,0.0,3.5,3.3,2.9,97,6
254978,2021-02-01 02:00:00,0.0,3.6,3.3,2.8,95,5
254979,2021-02-01 03:00:00,0.0,3.6,3.4,3.0,97,4
254980,2021-02-01 04:00:00,0.0,3.6,3.4,3.1,97,3


In [64]:
weather_data = weather_data[:-1] # drop the last row from 01/03/2022
weather_data.tail()

,date,rain,temp,wetb,dewpt,rhum,wdsp
264403,2022-02-28 19:00:00,0.0,2.5,1.7,0.3,86,6
264404,2022-02-28 20:00:00,0.0,2.2,1.4,0.0,86,7
264405,2022-02-28 21:00:00,0.0,1.1,0.6,-0.3,90,5
264406,2022-02-28 22:00:00,0.0,0.0,-0.3,-1.0,94,6
264407,2022-02-28 23:00:00,0.0,0.2,-0.1,-0.7,94,6


<a id="2"></a>
# <p style="font-size:100%; text-align:left; color:#444444;">2- Feature Engineering</p>

## Hypothesis

Hourly trend: It might be a high demand for people commuting to work. Early morning and late evening can have different trend (cyclist) and low demand during 10:00 pm to 4:00 am.

Daily Trend: Users demand more bike on weekdays as compared to weekend or holiday.

Rain: The demand of bikes will be lower on a rainy day as compared to a sunny day. Similarly, higher humidity will cause to lower the demand and vice versa.

Temperature: In Ireland, temperature has positive correlation with bike demand.

Traffic: It can be positively correlated with Bike demand. Higher traffic may force people to use bike as compared to other road transport medium like car, taxi etc.



### New Features
- date (yyyy-mm-dd)
- month
- hour
- workingday
- peak
- holiday
- season
- battery_start
- battery_end
- path? (multi polygon)
- rental_duration


The number of rentals each hour will be aggregate later with a new feature `count`.

In [65]:
rentals_data = historical_data.drop(['harvesttime','ebikestateid'], axis=1).copy()
rentals_data[["lastgpstime", "lastrentalstart"]] = rentals_data[["lastgpstime", "lastrentalstart"]].apply(pd.to_datetime)

rentals_data = rentals_data.astype({'battery': np.int16}, errors='ignore') # errors ignore to keep missing values (not throwing error)

In [66]:
rentals_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1667841 entries, 0 to 1667840
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   bikeid           1667841 non-null  int64         
 1   battery          1625500 non-null  float64       
 2   bikeidentifier   1667841 non-null  int64         
 3   biketypename     1667841 non-null  object        
 4   ebikeprofileid   1667841 non-null  int64         
 5   isebike          1667841 non-null  bool          
 6   ismotor          1667841 non-null  bool          
 7   issmartlock      1667841 non-null  bool          
 8   lastgpstime      1667841 non-null  datetime64[ns]
 9   lastrentalstart  1667841 non-null  datetime64[ns]
 10  latitude         1667841 non-null  float64       
 11  longitude        1667841 non-null  float64       
 12  spikeid          1667841 non-null  int64         
dtypes: bool(3), datetime64[ns](2), float64(3), int64(4), obje

### Rentals information

- `coordinates`: converting latitude and longitude to an array to store a GeoJSON object *MultiPoint* 
- `start_battery`: getting the battery status when the rental started
- `lastgpstime`: new variable that will only store the last record when grouping rentals

In [67]:
def feat_eng(x):
    d = {}
    d['coordinates'] = x[['latitude','longitude']].values.tolist()
    d['start_battery'] = list(x['battery'])[-1] # get the first battery status (when rental started)
    d['lastgpstime'] = list(x['lastgpstime'])[0] # get the last gpstime (previously sorted)
    
    return pd.Series(d, index=['coordinates', 'start_battery', 'lastgpstime'])

# also sorting data by lastgpstime
grouped_rentals = rentals_data.sort_values("lastgpstime", ascending=False).groupby(['lastrentalstart', 'bikeid']).apply(feat_eng).reset_index()

In [68]:
grouped_rentals.shape

(48166, 5)

### Date and time - new features
- `rental_date`
- `rental_month`
- `rental_hour`
- `holiday`
- `workingday`
- `peak`
- `season`: (1 = Spring, 2 = Summer, 3 = Fall, 4 = Winter)
- `duration`*: duration of the rental

\* **Assumption**: Due to lack of information and data, to calculate the average rent time I am assuming that when a new bike rental starts the average will be calculated by: $ ( AvgRentTime* = LastGPSTime - LastRentalStart ) $

In [69]:
grouped_rentals['rental_date'] = pd.to_datetime(grouped_rentals['lastrentalstart'].dt.date)
grouped_rentals['rental_hour'] = grouped_rentals['lastrentalstart'].dt.hour
grouped_rentals['rental_day'] = grouped_rentals['lastrentalstart'].dt.day
grouped_rentals['rental_month'] = grouped_rentals['lastrentalstart'].dt.month
grouped_rentals['rental_year'] = grouped_rentals['lastrentalstart'].dt.year

Slicing the dataset to get the sample as per weather data above.

In [70]:
start_date_hist, end_date_hist

(datetime.datetime(2021, 2, 1, 0, 0), datetime.datetime(2022, 3, 1, 0, 0))

In [71]:
grouped_rentals = grouped_rentals[(grouped_rentals.rental_date >= start_date_hist) & (grouped_rentals.rental_date <= end_date_hist)]

In [72]:
grouped_rentals.tail()

,lastrentalstart,bikeid,coordinates,start_battery,lastgpstime,rental_date,rental_hour,rental_day,rental_month,rental_year
48161,2022-02-27 20:16:55,48,"[[53.3624, -6.23703], [53.3624, -6.23702], [53...",33.0,2022-02-28 00:17:57,2022-02-27,20,27,2,2022
48162,2022-02-27 21:54:06,109,"[[53.3051, -6.21994], [53.3051, -6.21993], [53...",60.0,2022-02-28 00:05:03,2022-02-27,21,27,2,2022
48163,2022-02-27 21:57:44,49,"[[53.3119, -6.27482], [53.3119, -6.2748], [53....",37.0,2022-02-28 00:25:27,2022-02-27,21,27,2,2022
48164,2022-02-27 22:29:41,87,"[[53.3362, -6.2734], [53.3362, -6.27334], [53....",71.0,2022-02-28 00:05:19,2022-02-27,22,27,2,2022
48165,2022-02-27 22:29:42,10,"[[53.3495, -6.24842], [53.3495, -6.2484], [53....",78.0,2022-02-28 00:06:11,2022-02-27,22,27,2,2022


## Rental's duration

In [73]:
# time of rental in minutes (lastgpstime - rental-start)
grouped_rentals['duration'] = (grouped_rentals['lastgpstime'] - grouped_rentals['lastrentalstart']) / pd.Timedelta(minutes=1)

A few GPS records have frozen and stopped sending the accurate data back. About 345 records which would lead to a bias duration of rentals.

To prevent any inaccurate information these records will be set as `NaN`.

In [74]:
grouped_rentals['duration'] = np.where(grouped_rentals['duration'] < 0, np.NaN, grouped_rentals['duration'])
len(grouped_rentals[ np.isnan(grouped_rentals['duration']) ])

304

## Bank Holidays

In [75]:
bank_holidays = pd.read_json('../data/processed/irishcalendar.json')
bank_holidays['date'] = pd.to_datetime(arg=bank_holidays['date'],utc=True, infer_datetime_format=True)
bank_holidays['dt'] = pd.to_datetime(bank_holidays['date'].dt.date)

In [76]:
qry_bh = {
    'type': 'National holiday'
}

# bank_holidays = read_mongo(query=qry_bh, collection='irishcalendar')
bank_holidays.drop(['country', 'type'], axis=1, inplace=True)
# bank_holidays['date'] = pd.DatetimeIndex(bank_holidays['date'].apply(pd.to_datetime))

In [77]:
# holiday
grouped_rentals['holiday'] = grouped_rentals['rental_date'].isin(bank_holidays['dt'])

# day of the week
grouped_rentals['dayofweek_n'] = grouped_rentals['rental_date'].dt.dayofweek
grouped_rentals['dayofweek'] = grouped_rentals['rental_date'].dt.day_name()

# working day (Monday=0, Sunday=6)
# from 0 to 4 or monday to friday and is not holiday
grouped_rentals['working_day'] = grouped_rentals['dayofweek_n'] < 5
# set working_day to False on National Bank Holodays
grouped_rentals.loc[ grouped_rentals['holiday'] , 'working_day'] = False

## Seasons

In [78]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [('Winter', (datetime(Y,  1,  1),  datetime(Y,  3, 20))),
           ('Spring', (datetime(Y,  3, 21),  datetime(Y,  6, 20))),
           ('Summer', (datetime(Y,  6, 21),  datetime(Y,  9, 22))),
           ('Autumn', (datetime(Y,  9, 23),  datetime(Y, 12, 20))),
           ('Winter', (datetime(Y, 12, 21),  datetime(Y, 12, 31)))]

def get_season(date: pd.DatetimeIndex) -> str:
    '''
        Receives a date and returns the corresponded season
        0 - Spring | 1 - Summer | 2 - Autumn | 3 - Winter
        Vernal equinox(about March 21): day and night of equal length, marking the start of spring
        Summer solstice (June 20 or 21): longest day of the year, marking the start of summer
        Autumnal equinox(about September 23): day and night of equal length, marking the start of autumn
        Winter solstice (December 21 or 22): shortest day of the year, marking the start of winter
    '''
    date = date.replace(year=Y)
    return next(season for season, (start, end) in seasons if start <= date <= end)


grouped_rentals['season'] = grouped_rentals.rental_date.map(get_season)

In [79]:
grouped_rentals.groupby('season').size()

season
Autumn    7609
Spring    9268
Summer    9373
Winter    9353
dtype: int64

In [80]:
grouped_rentals.isnull().sum()

lastrentalstart      0
bikeid               0
coordinates          0
start_battery      548
lastgpstime          0
rental_date          0
rental_hour          0
rental_day           0
rental_month         0
rental_year          0
duration           304
holiday              0
dayofweek_n          0
dayofweek            0
working_day          0
season               0
dtype: int64

## Battery

In [81]:
grouped_rentals['start_battery'] = pd.to_numeric(grouped_rentals['start_battery'])

In [82]:
grouped_rentals[grouped_rentals['start_battery'] > 100]

,lastrentalstart,bikeid,coordinates,start_battery,lastgpstime,rental_date,rental_hour,rental_day,rental_month,rental_year,duration,holiday,dayofweek_n,dayofweek,working_day,season
18593,2021-04-03 12:40:00,103,"[[53.3405, -6.2679]]",268.0,2021-04-03 12:55:11,2021-04-03,12,3,4,2021,15.183333,False,5,Saturday,False,Spring


From the battery records there is a few cases that we can consider. Only one record has ` battery > 100` and a few negatives ones. To simplify the analysis the records will be normalized with values between `0 > x > 100`.

All missing values (*n=571*) will not be transformed as it could be only malfunction issue when transmiting the data and it could mislead the analysis.

In [83]:
# normalize battery status between 0 > x < 100
grouped_rentals['start_battery'] = abs(grouped_rentals['start_battery'])
grouped_rentals.loc[grouped_rentals['start_battery'] > 100, 'start_battery'] = 100

## Peak Times

>https://www.independent.ie/irish-news/the-new-commuter-hour-peak-times-increase-with-record-traffic-volumes-36903431.html

In [84]:
grouped_rentals['peak'] = grouped_rentals[['rental_hour', 'working_day']] \
    .apply(lambda x: (False, True)[(x['working_day'] == 1 and (6 <= x['rental_hour'] <= 10 or 15 <= x['rental_hour'] <= 19))], axis = 1)

## Times of the Day

- Morning (from 7am to noon)
- Afternoon (from midday to 6pm)
- Evening (from 6pm to 10pm)
- Night (from 10pm to 5am)

In [85]:
conditions = [
    (grouped_rentals['rental_hour'] < 7), # night
    (grouped_rentals['rental_hour'] >= 7) & (grouped_rentals['rental_hour'] < 12), # morning
    (grouped_rentals['rental_hour'] >= 12) & (grouped_rentals['rental_hour'] < 18), # afternoon
    (grouped_rentals['rental_hour'] >= 18) & (grouped_rentals['rental_hour'] < 23) # evening
]
values = ['Night', 'Morning', 'Afternoon', 'Evening']
grouped_rentals['timesofday'] = np.select(conditions, values,'Night')

In [86]:
new_rentals = grouped_rentals.copy()
new_rentals.to_csv('../data/interim/new_features_rentals.csv', index=False)

## Humidity

Attempt to create a new feature called `Humidity` to avoid Multicollinearity.

### We need different humidity quantities

The problem with relative humidity is that, by itself, it doesn’t really tell you how humid it is.

- **Relative Humidity** – This quantity tells us how close the conditions are to saturation, when condensation of water vapor can occur. The interaction of porous materials with water vapor increases with increasing RH. The chance of growing mold increases with increasing RH, 70% usually given as the threshold to stay below.
- **Dew Point Temperature** – This temperature scales with the amount of water vapor. As more water vapor enters a volume, the dew point goes up. If the air in your crawl space, for example, has a dew point of 75° F, you’re probably going to find condensation somewhere. Look at the water pipes, poorly insulated ducts, and uninsulated duct boots.
- **Wet Bulb Temperature** – If dew point is the temperature of condensation, wet bulb is the temperature of evaporation. Same concept; different direction. This one’s important for cooling our bodies.

Once you get a handle on these three quantities, you’ll have a pretty good understanding of humidity.

> https://www.energyvanguard.com/blog/problem-with-relative-humidity

In [87]:
weather_data['rhum'] = pd.to_numeric(weather_data['rhum'],errors = 'coerce')
weather_data['wetb'] = pd.to_numeric(weather_data['wetb'],errors = 'coerce')
weather_data['dewpt'] = pd.to_numeric(weather_data['dewpt'],errors = 'coerce')

In [88]:
weather_data[weather_data.rhum.isnull()]

,date,rain,temp,wetb,dewpt,rhum,wdsp


### Variance Inflation Factor(VIF)

In [89]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [90]:
# Fill NaN values: 'ffill' -> propagate last valid observation forward to next valid
weather_data.fillna(method="ffill", inplace=True) 

In [91]:
test_vif = weather_data.copy().drop('date', axis=1)
#Calculate VIF for each variable in the new data frame
vif = pd.DataFrame()
vif["features"] = test_vif.columns
vif["vif_value"] = [variance_inflation_factor(test_vif.values, i) for i in range(test_vif.shape[1])]
vif

,features,vif_value
0,rain,1.088090
1,temp,1309.806894
2,wetb,3378.828670
3,dewpt,463.724882
4,rhum,9.756297
5,wdsp,5.165330


To avoid multicolinearity we could create a new feature based on some calculation on those three features related to humidity. Unfortunately, the Forecast API that will be used in production **does not include Wet Bulb Temperature**.

There are a few equations that could applied to calculate Wet Bulb Temperature (eg: Stull formula) but because we do not have which one is used to store this feature on their historical the best would be not use this feature at all.

In [92]:
#weather_data.drop(['rhum', 'wetb', 'dewpt'], axis=1, inplace=True)
weather_data.drop(['wetb', 'dewpt'], axis=1, inplace=True)

## Combining Rentals and Weather data

In [93]:
rentals = new_rentals.copy()
weather = weather_data.copy()

weather['rental_date'] = pd.to_datetime(weather['date'].dt.date)
weather['rental_hour'] = weather['date'].dt.hour

In [94]:
all_data = pd.merge(rentals, weather, on=['rental_date', 'rental_hour'])
all_data.to_csv('../data/interim/all_data.csv', index=False)

## Grouping data to reflect hourly count of rentals

In [95]:
hourly_rentals = all_data.copy()
count_hourly_rentals = hourly_rentals.groupby(['rental_date', 'rental_hour']).size().reset_index(name='count')
columns_to_drop = ['lastrentalstart','bikeid','coordinates','start_battery','lastgpstime','rental_month','duration','date']
hourly_rentals = hourly_rentals.drop(columns_to_drop, axis=1).drop_duplicates(subset=['rental_date', 'rental_hour'])
hourly_rentals.shape, count_hourly_rentals.shape

((7447, 15), (7447, 3))

In [96]:
hourly_data = pd.merge(hourly_rentals, count_hourly_rentals, on=['rental_date','rental_hour'])
hourly_data.to_csv('../data/interim/hourly_data.csv', index=False)
hourly_data.head()

,rental_date,rental_hour,rental_day,rental_year,holiday,dayofweek_n,dayofweek,working_day,season,peak,timesofday,rain,temp,rhum,wdsp,count
0,2021-02-01,6,1,2021,False,0,Monday,True,Winter,True,Night,0.0,3.4,98,3,1
1,2021-02-01,8,1,2021,False,0,Monday,True,Winter,True,Morning,0.0,3.5,93,4,2
2,2021-02-01,9,1,2021,False,0,Monday,True,Winter,True,Morning,0.0,2.6,93,2,4
3,2021-02-01,10,1,2021,False,0,Monday,True,Winter,True,Morning,0.0,4.1,97,4,3
4,2021-02-01,11,1,2021,False,0,Monday,True,Winter,False,Morning,0.0,5.2,86,6,12


<img title="GitHub Mark" src="./img/GitHub-Mark-64px.png" style="height: 32px; padding-right: 15px" alt="GitHub Mark" align="left"> [GitHub repository](https://github.com/pessini/moby-bikes) <br>Author: Leandro Pessini